### Requirement
1. Install ```image-matching-models```: https://github.com/gmberton/image-matching-models

In [ ]:
import sys
from matching import get_matcher, available_models
from matching.viz import *
from pathlib import Path
import torch
import warnings

warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2
%matplotlib inline


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')
ransac_kwargs = {'ransac_reproj_thresh':5, 
                 'ransac_conf':0.995, 
                 'ransac_iters':1000} # optional ransac params
matcher = get_matcher('master', device=device, **ransac_kwargs) #try an ensemble!

In [ ]:
pair_path = '/Rocket_ssd/dataset/data_litevloc/general_environment/map_free_eval/test/example_pairs_paths_all_scene.txt'
resize = 512
lines = []
with open(pair_path, 'r') as file:
    for line in file:
        if line == '\n':
            continue
        lines.append(line)
print(len(lines))

### Match Image Pair

In [ ]:
for i in range(1, len(lines), 1):
    pair = lines[i].split()
    img0 = matcher.load_image(pair[0], resize=resize)
    img1 = matcher.load_image(pair[1], resize=resize)
    result = matcher(img0, img1)
    num_inliers, H, mkpts0, mkpts1 = result['num_inliers'], result['H'], result['inlier_kpts0'], result['inlier_kpts1']
    plot_matches(img0, img1, result, save_path=f'/Rocket_ssd/dataset/data_litevloc/general_environment/map_free_eval/test/result_matching/{i:06d}.png')

### Extract Kpts and Descriptors

In [ ]:
pair_path = '/Rocket_ssd/dataset/data_litevloc/general_environment/map_free_eval/test/example_pairs_paths_all_scene.txt'
resize = 512
with open(pair_path, 'r') as file:
    for line in file:
        pair = line.split()
        img = matcher.load_image(pair[1], resize=resize)
        result = matcher.extract(img)
        plot_kpts(img, result)

### See all matchers on the same image pair

In [ ]:
%%capture

image_size = 512
img0 = matcher.image_loader(pair[0], resize=image_size)
img1 = matcher.image_loader(pair[1], resize=image_size)
num_per_row = 3
fig, axs = plt.subplots(int(np.ceil(len(available_models)/num_per_row)), 2*num_per_row, figsize=(12, 20))
flat_axs = axs.flatten()

for idx, model in enumerate(available_models):
    idx = idx * 2
    ax0, ax1 = flat_axs[idx], flat_axs[idx+1]

    try:
        matcher = get_matcher(model, device=device)
        result = matcher(img0, img1)
        num_inliers, H, mkpts0, mkpts1 = result['num_inliers'], result['H'], result['inlier_kpts0'], result['inlier_kpts1']

        ax0.imshow(tensor_to_image(img0))
        ax1.imshow(tensor_to_image(img1))
        _ = viz2d.plot_matches(mkpts0, mkpts1, color='lime', lw=0.1, ps=1, axes=[ax0, ax1])
        ax0.text(image_size/2, -10, f'{model}: {len(mkpts1)} matches')

    except Exception as e:
        blank_img = np.zeros_like(tensor_to_image(img0))
        ax0.imshow(blank_img)
        ax1.imshow(blank_img)
        ax0.text(image_size/2, -10, f'{model}: Error')
        print(f'Error with {model}: {e}')
    
for ax in flat_axs:
    ax.axis('off')

In [ ]:
fig